In [ ]:
import pandas as pd
import os

# GF TODO: check if excel present, otherwise download it
# wget http://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

excel_with_path  = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.xlsx'
pickle_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.pkl'
csv_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.csv'

df = None

if os.path.isfile(pickle_with_path):
    print('-> picke already exists, much faster\n')
    df = pd.read_pickle(pickle_with_path)
else:
    print('-> picke does not exist, go to excel, and create it\n')
    df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
    df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
    df = pd.concat([df1, df2])
    df.to_pickle(pickle_with_path)

df['Transaction'] = df.Quantity * df.Price
df=df.rename(columns={"Customer ID": "Customer_ID"})

# df.to_csv(csv_with_path, encoding = 'utf-8')

# GF TODO: there are nan 
# GF TODO: there are fields which are 'empty char', e.g. in the customer_id

In [ ]:
df.head(5)

## What do I want to extract from this dataset?
- https://en.wikipedia.org/wiki/Exploratory_data_analysis
- how many customers
- how many customer in which country
- how much do customers spend => make a PLOT!
- trends of spending: overall, by the largest customer, by the smaller customers

- how many different types of items
- what kind of items are bought the most (by NUMBER of by REVENUE), are cancelled the most, 
- how much revenue per type of item bought
- correlation between 

- cust.groupby('customer_unique_id').size().value_counts() => The majority of customers made only a single purchase.


##  Are there users at high risk of churning by the end of 2011

- https://www.kdnuggets.com/2019/05/churn-prediction-machine-learning.html 
- https://towardsdatascience.com/hands-on-predict-customer-churn-5c2a42806266
- https://www.google.com/search?q=dwarf+on+the+shoulders+of+giants&rlz=1C5CHFA_enCH771CH771&sxsrf=ALeKk034vLLmAHn5V1c0QAON4DZuO1GqVA:1582207255390&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiRrayrpeDnAhXCxaYKHZQVCgcQ_AUoAXoECBIQAw&biw=1418&bih=915#imgrc=oj4GwW0aHYfVYM
- similar exercise resolved here https://towardsdatascience.com/modeling-customer-churn-for-an-e-commerce-business-with-python-874315e688bf
- the repo of the library to be used https://github.com/CamDavidsonPilon/lifetimes
- the reference paper in pdf http://brucehardie.com/papers/018/fader_et_al_mksc_05.pdf http://brucehardie.com/papers/bgnbd_2004-04-20.pdf and the journal which has all aspects of a respectable journals 
- GF TODO: once you've found which customers have churned, make a few hitory plots to prove that indeed they have churned, to show that I am not blindly trusing the package I've downloade

In [ ]:
print('There are %d unique customers'%len(df.Customer_ID.unique()))

In [ ]:
print('Customers are from %d countries'%len(df.Country.unique()))

In [ ]:
# GF can be removed
# df.groupby('Country').Customer_ID.unique()

In [ ]:
print('91 % of the customers are from the UK, 2% from Germany, 1.5% are from France. \
 The remaining 40 countries have less than 1% of the customers')

df.groupby('Country').Customer_ID.unique().agg([len])    \
    .rename(columns={"len": "num_customers"})            \
    .sort_values(by=['num_customers'],ascending=False)


In [ ]:
# the largest customer by total net revenue (i.e. taking into account cancellations)
# is in the UK and has ID 18102.0
df.groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                  \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Total Customer Revenue'],ascending=False).head(10)        

In [ ]:
df.groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                  \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Number of Transaction'],ascending=False).head(10)    

In [ ]:
# the customer with the largest number of transactions is 
# is in the UK and has ID 17841.0
df.loc[df.Transaction>0]                                                                                      \
    .groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Number of Transaction'],ascending=False).head(10)    

In [ ]:
# the UK generates the largest revenue (16M over the two years),
# surprisingly EIRE is the second largest source of revenue, with 0.6M (despite having only 6 customers, they must be huge customers)
df.groupby('Country').Transaction.sum().sort_values(ascending=False)

In [ ]:
df.groupby('Country').Transaction.sum().sort_values(ascending=False)

In [ ]:
# pip install --user lifetimes
from lifetimes.utils import *


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from lifetimes.utils import *
from lifetimes import BetaGeoFitter,GammaGammaFitter
from lifetimes.plotting import plot_probability_alive_matrix, plot_frequency_recency_matrix, plot_period_transactions, plot_cumulative_transactions,plot_incremental_transactions
from lifetimes.generate_data import beta_geometric_nbd_model
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions,plot_history_alive

In [ ]:
print('pip install worked out fine')